# Hyperparameter Tuning using HyperDrive

In the cell below, we import all the dependencies that we need to complete the project.

In [20]:
from azureml.core import Workspace, Dataset, ComputeTarget, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.model_selection import train_test_split
from azureml.core import Model
from azureml.core import Environment
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.widgets import RunDetails

In [21]:
ws = Workspace.from_config()


experiment_name = 'capstone-project-hyper'
experiment=Experiment(ws, experiment_name)

run = experiment.start_logging()

In [22]:
cluster_name = "compute-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print(f"Found existing compute target: {compute_target}")
except Exception as e:
    print(f"Creating a new compute target (error: {e}")
    compute_cnfg = AmlCompute.provisioning_configuration(
        vm_size = "Standard_DS3_V2",
        min_nodes = 0,
        max_nodes = 4,
    )
    compute_target = ComputeTarget.create(
        ws,
        cluster_name,
        compute_cnfg,
    )
    compute_target.wait_for_completion(
        show_output=True,min_node_count=None,timeout_in_minutes=10,
    )


print(f'compute target: {compute_target.get_status().serialize()}')

Found existing compute target: AmlCompute(workspace=Workspace.create(name='quick-starts-ws-262203', subscription_id='a24a24d5-8d87-4c8a-99b6-91ed2d2df51f', resource_group='aml-quickstarts-262203'), name=compute-cluster, id=/subscriptions/a24a24d5-8d87-4c8a-99b6-91ed2d2df51f/resourceGroups/aml-quickstarts-262203/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-262203/computes/compute-cluster, type=AmlCompute, provisioning_state=Succeeded, location=westeurope, tags={})
compute target: {'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2024-07-11T05:09:10.166000+00:00', 'errors': None, 'creationTime': '2024-07-11T04:18:26.716450+00:00', 'modifiedTime': '2024-07-11T04:18:36.735884+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTim

## Dataset

### Overview

We used heart failure dataset(https://www.kaggle.com/datasets/andrewmvd/heart-failure-clinical-data). Downloaded this dataset from the kaggle website and uploaded to Azure ML Studio and registered it. This data generally contains data about people and has health related data with the death column. Generally, we have to predict early detection of Heart rate failure using ML.

Columns in dataset are - age,	anaemia	creatinine_phosphokinase,	diabetes,	ejection_fraction,	high_blood_pressure	platelets,	serum_creatinine,	serum_sodium,	sex	smoking	time,	DEATH_EVENT.

Heart failure is a common event caused by CVDs and this dataset contains 12 features that can be used to predict mortality by heart failure.
Reason for ML - Need early detection and management wherein a machine learning model can be of great help.

In [23]:
dataset_name = 'heart-beat'
dataset = Dataset.get_by_name(workspace=ws, name=dataset_name)

In [24]:
# inspect the dataframe 
df = dataset.to_pandas_dataframe()
df.head()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## Hyperdrive Configuration

RandomParameterSampling - This is a strategy used to randomly sample hyperparameters for model training. During the experiment, the ML framework will randomly select values from the specified choices for each hyperparameter.

learning_rate: The rate at which the model learns during training. The choices given are 0.01, 0.05, 0.1, 0.25, 1.0.
n_estimators: The number of trees in an ensemble model (like a Random Forest or Gradient Boosting). The choices are 1, 5, 10, 25.

BanditPolicy = This is an early stopping policy that helps in terminating poorly performing runs early, based on performance compared to the best run.

evaluation_interval: Specifies how often (in terms of number of iterations) the policy should evaluate the performance of the runs. Here, it is set to 3, meaning the performance will be checked every 3 iterations.
slack_factor: This sets the tolerance level for how much worse a run can perform relative to the best run before being terminated. A slack_factor of 0.2 means that a run can perform up to 20% worse than the best performing run before it is stopped.

In this project we used Gradient boosting as it builds an ensemble of trees sequentially, where each tree tries to correct the errors of the previous one. This iterative process often results in high predictive accuracy.  It is especially useful when you need a model that can capture complex patterns in the data while still being interpretable and customizable.



In [25]:
ps = RandomParameterSampling({
    "learning_rate": choice(0.001,0.01,0.02, 0.05, 0.1, 0.25 ,0.75 , 1.0),
    "n_estimators": choice(1,3 , 5,7, 10,15,20, 25)
})

import os

policy = BanditPolicy(
    evaluation_interval=5,
    slack_factor=0.3)

if "capstone-file-hyper" not in os.listdir():
    os.mkdir("./capstone-file-hyper")


sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependecies.yml')

src = ScriptRunConfig(
    source_directory="./",
    script="train-heart-beat-dataset.py",
    compute_target=compute_target,
    environment=sklearn_env)

hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=17,
    max_concurrent_runs=4)


In [26]:
hyperdrive_run = experiment.submit(hyperdrive_config)

## Run Details

In the cell below, we use the `RunDetails` widget to show the results of the hyperdrive experiment defined above and look at the best model as well as its properties.

In [27]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_15e83d28-d264-44e2-b7db-3393e1c887b8
Web View: https://ml.azure.com/runs/HD_15e83d28-d264-44e2-b7db-3393e1c887b8?wsid=/subscriptions/a24a24d5-8d87-4c8a-99b6-91ed2d2df51f/resourcegroups/aml-quickstarts-262203/workspaces/quick-starts-ws-262203&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-07-11T05:21:24.513454][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2024-07-11T05:21:25.1479209Z][SCHEDULER][INFO]Scheduling job, id='HD_15e83d28-d264-44e2-b7db-3393e1c887b8_0' 
[2024-07-11T05:21:25.2069596Z][SCHEDULER][INFO]Scheduling job, id='HD_15e83d28-d264-44e2-b7db-3393e1c887b8_1' 
[2024-07-11T05:21:25.3696594Z][SCHEDULER][INFO]Scheduling job, id='HD_15e83d28-d264-44e2-b7db-3393e1c887b8_2' 
[2024-07-11T05:21:25.4708936Z][SCHEDULER][INFO]Scheduling job, id='HD_15e83d28-d264-44e2-b7db-3393e1c887b8_3' 
[2024-07-11T05:21:25.326780][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

{'runId': 'HD_15e83d28-d264-44e2-b7db-3393e1c887b8',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2024-07-11T05:21:23.198463Z',
 'endTimeUtc': '2024-07-11T05:33:00.091287Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '180e3fca-11c9-44b3-b9c4-f168d4645d27',
  'user_agent': 'python/3.9.19 (Linux-5.15.0-1064-azure-x86_64-with-glibc2.31) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.56.0',
  'space_size': '64',
  'best_child_run_id': 'HD_15e83d28-d264-44e2-b7db-3393e1c887b8_9',
  'score': '0.7666666666666667',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_15e83d28-d264-44e2-b7db-3393e1c887b8_9'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlCl

Current provisioning state of AmlCompute is "Deleting"



## Best Model

In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [28]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
best_run_parameter_values = best_run.get_details()["runDefinition"]["arguments"]

print(f"Best Model Run ID: {best_run.id}\n")
print(f"Best Model Metrics: {best_run_metrics}\n")
print(f"Best Model Accuracy: {best_run_metrics['Accuracy']}\n")


Best Model Run ID: HD_15e83d28-d264-44e2-b7db-3393e1c887b8_9

Best Model Metrics: {'Learning Rate:': 0.75, 'Number Estimators:': 7, 'Accuracy': 0.7666666666666667}

Best Model Accuracy: 0.7666666666666667



## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.


In [29]:
model = best_run.register_model(model_path="trained_model.pkl",model_name="best_model_gradient_boosting",
                               properties={
                                   "Accuracy": best_run_metrics["Accuracy"],
                                   "Number Estimators": best_run_metrics["Number Estimators:"],
                                   "Learning Rate": best_run_metrics["Learning Rate:"]
                               })

In [30]:

compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

